In [1]:
from google.colab import drive
drive.mount("/content/drive",force_remount=True)

Mounted at /content/drive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri May 28 00:37:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os 
os.chdir('drive/MyDrive/OASIS_Data')

In [4]:
import copy 
import time
import datetime
import numpy as np 
import pandas as pd 
import nibabel as nib
import sklearn as skl
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [5]:
import torch 
from torch.utils.data import Dataset, DataLoader
from torch import nn 
from torch import optim

In [6]:
pip install torchio nilearn

     |████████████████████████████████| 153kB 2.1MB/s 
     |████████████████████████████████| 3.1MB 16.5MB/s 
     |████████████████████████████████| 42.5MB 121kB/s 


In [7]:
import torchio as tio 

In [8]:
os.chdir("MRI_SOMVAE")
from OASISDataset import *
from FullModels import *
from TrainingTesting import *
from ExtraMetrics import *
from RegularSOM import *
os.chdir("..")

Split into Train and Val (same seed as before)

In [10]:
fulltrainlist = pd.read_csv("oasis_ctrl_training.csv")

trainlist,vallist = train_test_split(fulltrainlist, random_state=128, test_size=0.15)

folder = "ctrl_original_resolution"

In [11]:
start = time.time()
traindataset = OASISDataset_Colab(folder,trainlist)
print(time.time()-start)
valdataset = OASISDataset_Colab(folder,vallist)
print(time.time()-start)

365.68765234947205
428.8333113193512


In [12]:
batch_size=32

train_dataloader = DataLoader(traindataset,batch_size=batch_size,shuffle=True)
val_dataloader = DataLoader(valdataset,batch_size=len(valdataset),shuffle=True)

Load Regular Autoencoder model

In [13]:
AEcheckpoint = torch.load("VanillaAE32_Run2_4filters_051921.tar")

RegularAE = VanillaAE(num_channels=1,embedding_dim=32,num_filters=4,batchnorm=True)
RegularAE.load_state_dict(AEcheckpoint["model_state_dict"])

<All keys matched successfully>

In [14]:
RegularAE.eval()

VanillaAE(
  (encoder): VQEncoder(
    (conv1): Sequential(
      (0): Conv3d(1, 4, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
      (1): BatchNorm3d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv2): Sequential(
      (0): Conv3d(4, 8, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
      (1): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv3): Sequential(
      (0): Conv3d(8, 16, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
      (1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv4): Sequential(
      (0): Conv3d(16, 32, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
    )
  )
  (decoder): VQDecoder(
    (conv1): Sequential(
      (0): ConvTranspose3d(32, 16, kernel_size=(4, 4, 4), stride=(2, 2, 2), padding=(1, 1, 1))
      (1): BatchNorm3d(16, ep

Get Encoder Output 

In [15]:
Xenc = encoder_output(RegularAE,train_dataloader)

In [19]:
RSOM = RegularSOM(num_embeddings=256,embedding_dim = 32,som_h=16,som_w=16)

In [20]:
losses = train_SOM(Xenc,RSOM,500)


Epoch 0 Complete. Average Training Loss: 340.86993408203125 
Epoch 1 Complete. Average Training Loss: 319.54010009765625 
Epoch 2 Complete. Average Training Loss: 322.4495544433594 
Epoch 3 Complete. Average Training Loss: 331.5652770996094 
Epoch 4 Complete. Average Training Loss: 329.57147216796875 
Epoch 5 Complete. Average Training Loss: 334.0205078125 
Epoch 6 Complete. Average Training Loss: 325.2816162109375 
Epoch 7 Complete. Average Training Loss: 324.754638671875 
Epoch 8 Complete. Average Training Loss: 316.4796447753906 
Epoch 9 Complete. Average Training Loss: 316.1865234375 
Epoch 10 Complete. Average Training Loss: 314.6938781738281 
Epoch 11 Complete. Average Training Loss: 325.78350830078125 
Epoch 12 Complete. Average Training Loss: 314.3992919921875 
Epoch 13 Complete. Average Training Loss: 311.0179443359375 
Epoch 14 Complete. Average Training Loss: 323.1549987792969 
Epoch 15 Complete. Average Training Loss: 329.70794677734375 
Epoch 16 Complete. Average Training 

In [32]:
torch.save({'embeddings': RSOM._embedding,'losses': losses},"RegularSOM_500epochs_embeddings.tar")

In [34]:
emb = RSOM._embedding.weight.detach().numpy()

In [40]:
re_emb = emb.reshape(16,16,-1)

In [42]:
re_emb[1,0,:]

array([ 2.62234998e+00, -2.06268120e+00,  7.26296604e-01,  3.20984554e+00,
        1.21661544e-01, -5.68044901e-01,  1.33854449e-02,  1.42590547e+00,
       -1.24985528e+00, -5.21209836e-01,  2.32894683e+00,  2.13625336e+00,
        3.60052288e-01,  2.52040577e+00, -3.39386702e+00, -2.94612765e-01,
       -2.14843917e+00, -2.99391007e+00,  1.20950413e+00, -1.51320291e+00,
        1.56296253e-01, -1.48033190e+00, -1.66516662e-01, -1.64300859e-01,
        5.59161186e-01,  1.74287748e+00,  3.30872202e+00, -6.20120764e-01,
       -1.31495142e+00, -3.42698669e+00, -1.55051908e+01,  4.63625550e-01],
      dtype=float32)

In [43]:
re_emb[1,1,:]

array([ 0.550047  , -0.0272268 , -0.02620049, -1.4419414 , -0.6706929 ,
        1.3875611 ,  0.8102162 ,  1.3493413 ,  0.24081227, -0.38968945,
        1.4002951 ,  0.7793142 , -0.04746952, -0.48314664, -0.8985255 ,
        1.6150533 ,  0.9855737 , -1.7032428 ,  0.5294519 ,  0.24115485,
        0.20629922,  1.153804  ,  0.10937766, -0.7000751 , -1.8163986 ,
        0.00744709, -2.1807647 , -0.5612849 , -1.1929904 , -1.0332062 ,
       -0.70237446,  0.76600474], dtype=float32)